In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import numpy as np
import math
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source
from IPython.display import SVG

In [2]:
class PathResolver:
    _relative_directory = '..'
    _source_directory = '{}/Src'.format(_relative_directory)
    _data_directory = '{}/Data'.format(_relative_directory)

    def GetDataFilepath(__self__, filename):
      return '{}/{}'.format(__self__._data_directory, filename)

    def GetSrcFilepath(__self__, filename):
      return '{}/{}'.format(__self__._source_directory, filename)

    def ReadDataFrameFromFilepath(__self__, filename):
      return pd.read_csv(__self__.GetDataFilepath(filename), sep=";")

In [3]:
path_resolver = PathResolver()

In [4]:
clients = path_resolver.ReadDataFrameFromFilepath('client.asc')
clients.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


In [5]:
class DateResolver:
    #Birthnumber is an abstraction that holds an encoded yymmdd in a specific format
    def resolve_gender_and_birthdate(__self__, date_number):
        [yy, mm, dd] = __self__._resolve_birthdate(date_number)
        
        return (__self__._resolve_gender_from_month(mm), __self__.get_date(yy, __self__._correct_month_if_necessary(mm), dd))
    
    def resolve_birthdate(__self__, date_number):
        [yy, mm, dd] = __self__._resolve_birthdate(date_number)
        
        return __self__.get_date(yy, __self__._correct_month_if_necessary(mm), dd)
    
    def _resolve_birthdate(__self__, date_number):
        pair_list = [int(x) for x in re.findall('\d{2}', str(date_number))]
        
        return [pair_list[0], pair_list[1], pair_list[2]]
        
    # If the month is larger than 50, then it is a woman (MM + 50)
    def _resolve_gender_from_month(__self__, mm):
        return 'Male' if mm < 50 else 'Female'
    
    def _correct_month_if_necessary(__self__, mm):
        return mm - 50 if mm >= 50 else mm
            
    def get_date(__self__, yy, mm, dd):
        return '19{}-{}-{}'.format(yy, mm, dd)

In [ ]:
reference_analysis_date = pd.to_datetime("1999-09-15")

def get_int_years_delta(start_date, end_date):
    return len(pd.date_range(start = start_date, end = end_date,freq='D')) // 365

In [ ]:
date_resolver = DateResolver()
genders_and_birth_dates = clients.birth_number.apply(date_resolver.resolve_gender_and_birthdate)
clients['gender'] = [gender for (gender, _) in genders_and_birth_dates]
clients['birth_date'] = [birth_date for (_, birth_date) in genders_and_birth_dates]
clients.birth_date = clients.birth_date.apply(pd.to_datetime)
clients['age'] = clients.birth_date.apply(lambda birth_date: get_int_years_delta(birth_date, reference_analysis_date))

clients.head()

In [ ]:
disposition = path_resolver.ReadDataFrameFromFilepath('disp.asc')
disposition.head()

In [ ]:
districts = path_resolver.ReadDataFrameFromFilepath('district.asc')
districts.head()

In [ ]:
districts.columns = ['district_id','district_name','region',"n_inhabitants",
                    "municip_less_499","municip_between_500_1999","municip_between_2000_9999",
                    "municip_more_10000", "n_cities","ratio_urban_inhab","avg_salary","unemploy_rate_95",
                    "unemploy_rate_96","n_entrepreneurs_1000_inhab","n_commited_crimes_95","n_commited_crimes_96"]
districts.head()

In [ ]:
clients_dispositions = pd.merge(clients, disposition, on = 'client_id')
clients_dispositions.head()

In [ ]:
credit_cards = path_resolver.ReadDataFrameFromFilepath('card.asc')
credit_cards.issued = pd.to_datetime(credit_cards.issued.apply(date_resolver.resolve_birthdate))
credit_cards.head()

In [ ]:
class CzechTranslator:    
    _cz_to_en_dict = {
        'POPLATEK MESICNE': 'monthly', 'POPLATEK TYDNE': 'weekly', 'POPLATEK PO OBRATU': 'after_transaction',
        'PRIJEM': 'credit', 'VYDAJ': 'debit', 'VYBER': 'choice',
        'VYBER KARTOU': 'card_debit', 'VKLAD': 'cash_credit', 'VYBER': 'cash_debit', 
        'PREVOD Z UCTU': 'other_bank_credit', 'PREVOD NA UCET': 'other_bank_debit',
        'POJISTNE': 'insurance', 'SIPO': 'household', 'LEASING': 'leasing', 'UVER': 'loan',
        'DUCHOD': 'pension', 'SLUZBY': 'statement_for_payment', 'SANKC. UROK': 'sanction_negative_balance', 'UROK': 'interest_credit'
    }
    
    def translate_if_possible(__self__, word):
        if type(word) != str:
            return ''
        
        uppercase_word = str.upper(word)
        
        return __self__._cz_to_en_dict[uppercase_word] if __self__._cz_to_en_dict.get(uppercase_word) != None else word
            
czech_to_en = CzechTranslator()

In [ ]:
# Translates czech words
orders = path_resolver.ReadDataFrameFromFilepath('order.asc')
orders.k_symbol = orders.k_symbol.apply(czech_to_en.translate_if_possible)
orders.head()

In [ ]:
# Translates A, B, C and D to human readable fields
loans_status_lookup = {'A': 'finished_ok', 'B': 'finished_with_default', 'C': 'running_ok', 'D': 'running_in_debt'}
loans = path_resolver.ReadDataFrameFromFilepath('loan.asc')
loans.date = pd.to_datetime(loans.date.apply(date_resolver.resolve_birthdate))
loans.status = [loans_status_lookup[status] for status in loans.status]
loans.head()

In [ ]:
accounts = path_resolver.ReadDataFrameFromFilepath('account.asc')
accounts.date = pd.to_datetime(accounts.date.apply(date_resolver.resolve_birthdate))
accounts.frequency = accounts.frequency.apply(czech_to_en.translate_if_possible)
print(accounts.frequency.value_counts())
accounts.head()

In [ ]:
transactions = pd.read_csv(path_resolver.GetDataFilepath('trans.asc'), sep=";", dtype = {'operation': str})
transactions.date = pd.to_datetime(transactions.date.apply(date_resolver.resolve_birthdate))
transactions.type = transactions.type.apply(czech_to_en.translate_if_possible)
print(transactions.type.value_counts())
print("==============================")
transactions.operation = transactions.operation.apply(czech_to_en.translate_if_possible)
print(transactions.operation.value_counts())
transactions.head()

In [ ]:
clients_districts = pd.merge(clients_dispositions, districts, on = 'district_id')
clients_districts.head()

In [ ]:
accounts_districts = pd.merge(accounts, districts, on = 'district_id')
accounts_districts.head()

In [ ]:
clients_accounts = pd.merge(clients_dispositions, accounts, on = 'account_id', suffixes = ('_client', '_account'))
clients_accounts.head()

In [ ]:
clients_districts_cards = pd.merge(clients_districts, credit_cards, on = 'disp_id', suffixes=('_client', '_card'))
clients_districts_cards.head()

In [ ]:
clients_accounts_loans = pd.merge(clients_accounts, loans, on = 'account_id', suffixes = ('_client', '_loan'))
clients_accounts_loans.head()

In [ ]:


# GetScoreForFactorVariables <- function(data, discript, strings)
# {
#   values <- data[, discript]
#   scores <- 1:length(strings)
#   names(scores) <- strings
#   data$scores <- scores[values]
  
#   return (data$scores)
# }

# GetScoreForNumericVariables <- function(data, posneg)
# {
#   values <- as.double(unlist(data))
#   score <- as.integer(NA)
#   data <- data.frame(values, score)
  
#   posneg = "p"
#   range1 <- between(data$values,quantile(data$values, na.rm = T)[1],quantile(data$values, na.rm = T)[2])
#   range2 <- between(data$values,quantile(data$values, na.rm = T)[2],quantile(data$values, na.rm = T)[3])
#   range3 <- between(data$values,quantile(data$values, na.rm = T)[3],quantile(data$values, na.rm = T)[4])
#   range4 <- between(data$values,quantile(data$values, na.rm = T)[4],quantile(data$values, na.rm = T)[5])
  
#   if(posneg == "p")
#   {
#     data$score <- ifelse(range1 == TRUE, 1,ifelse(range2 == TRUE, 2,ifelse(range3 == TRUE, 3, ifelse(range4 == TRUE, 4, NA))))
    
#     return(data$score)
    
#   }else if(posneg == "n")
#   {
    #data$score <- ifelse(range1 == TRUE, 4,ifelse(range2 == TRUE, 3,ifelse(range3 == TRUE, 2, ifelse(range4 == TRUE, 1, NA))))
    
#    return(data$score)
#  }else
#  {
#    print("Insira p ou n para obter o score desejado.")  
#  }      
#}  

#ConvertCharAndIntegerVariables <- function(data){
#  data[sapply(data, is.integer)] <- lapply(data[sapply(data, is.integer)], as.numeric)
#  data[sapply(data, is.character)] <- lapply(data[sapply(data, is.character)], as.factor) 
  
#  return(sapply(data, class))
#}

#Original R code to be rewritten
#clientrating.df <- clientsdisp.df[c(1,4,6,8)]
#clientrating.df$cl_district_id <- joinclientaccount$cl_district_id
#clientrating.df$acnt_district_id <- joinclientaccount$acnt_district_id
#clientrating.df$diff_district <- ifelse(clientrating.df$cl_district_id != clientrating.df$acnt_district_id, 1, 0)

#client district
#clientrating.df$dis_avg_salary <- GetScoreForNumericVariables(joinclientdistrict$avg_salary,"p")
#clientrating.df$dis_unemp_rate_95 <- GetScoreForNumericVariables(joinclientdistrict$unemploy_rate_95,"n")
#clientrating.df$dis_unemp_rate_96 <- GetScoreForNumericVariables(joinclientdistrict$unemploy_rate_96,"n")
#clientrating.df$dis_commit_crimes_95 <- GetScoreForNumericVariables(joinclientdistrict$n_commited_crimes_95,"n")
#clientrating.df$dis_commit_crimes_96 <- GetScoreForNumericVariables(joinclientdistrict$n_commited_crimes_96,"n")


# client type - Owner - 2, DISPONENT - 1
#clientrating.df$client_type <- as.numeric(revalue(joinclientdistrict$type_cl,replace = c("OWNER" = 2, "DISPONENT" = 1)))

# Card score - Gold - 3, Classic - 2, Junior - 1
#clientrating.df$card <- as.numeric(revalue(joinclientcard$type_card,replace = c("gold" = 3, "classic" = 2, "junior" = 1)))

# Years that the client has the card
#clientrating.df$year_card <- GetScoreForNumericVariables(GetAgeFromRefDate(joinclientcard$issued,refdate),"p")

# Account frequency
#clientrating.df$acnt_frequency <- GetScoreForFactorVariables(joinclientaccount, "frequency", c("Monthly", "Weekly", "AfterTransaction"))

# Years that the client has the account
#clientrating.df$year_account <- GetScoreForNumericVariables(GetAgeFromRefDate(joinclientaccount$date,refdate))

# Loans
#clientrating.df$loan_status <- GetScoreForFactorVariables(joinclientacntloans, "status", c("ContractFinishedDefault", "RunningInDebt", "RunningOk","ContractFinishedOk"))

#clientrating.df$year_loan <- GetAgeFromRefDate(joinclientacntloans$date_loan,refdate) 
#clientrating.df$duration_loan <- joinclientacntloans$duration           
#year_loan <- clientrating.df$year_loan * 12
#duration <- joinclientacntloans$duration
#clientrating.df$mth_to_pay_loan <- ifelse(year_loan < duration, duration - year_loan,0)


# sum score
#clientrating.df$sum_score <- rowSums(clientrating.df[,7:18], na.rm = TRUE) 

#colnames(clientrating.df)

In [ ]:
clients_rating = clients_dispositions[['client_id', 'gender', 'account_id']]
clients_accounts_loans.amount.quantile([0.0, 0.25, 0.5, 0.75, 1.0])

In [ ]:
# Still not using python processed data
arquivo = pd.read_csv("banco.csv", sep = ";")

arquivo.fillna(0, inplace = True)

y = arquivo["sum_scoreD"]
X = arquivo[["gender", "age", "dis_avg_salary", "dis_unemp_rate_95", "dis_unemp_rate_96",
        "dis_commit_crimes_95", "dis_commit_crimes_96", "client_type", "card", "year_card", "acnt_frequency", 
        "year_account", "loan_status"]]

dt = DecisionTreeClassifier(min_samples_split=1000, random_state=99)
dt.fit(X, y)

graph2 = Source(export_graphviz(dt, out_file=None
   , feature_names=["gender", "age", "dis_avg_salary", "dis_unemp_rate_95", "dis_unemp_rate_96",
        "dis_commit_crimes_95", "dis_commit_crimes_96", "client_type", "card", "year_card", "acnt_frequency", 
        "year_account", "loan_status"], class_names=['Good', 'Average', 'Ok', 'Bad'] 
   , filled = True))

display(SVG(graph2.pipe(format='svg')))